In [ ]:
import IPython.display
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.visualization
import named_arrays as na
import esis

In [ ]:
plt.rcParams["animation.embed_limit"] = 50

In [ ]:
astropy.visualization.quantity_support();

In [ ]:
level_0 = esis.flights.f1.data.level_0()

In [ ]:
unbiased = level_0.unbiased

In [ ]:
axis_time = level_0.axis_time
axis_channel = level_0.axis_channel
axis_x = level_0.axis_x
axis_y = level_0.axis_y
axis_xy = (axis_x, axis_y)
axis_txy = (axis_time, axis_x, axis_y)

In [ ]:
time = level_0.inputs.time
time = time.replace(ndarray=time.ndarray.datetime)

In [ ]:
fig, ax = plt.subplots(
    figsize=(6, 2.5),
    constrained_layout=True,
)
na.plt.plot(
    time,
    unbiased.outputs.mean(axis_xy),
    axis=axis_time,
    label=unbiased.channel,
    drawstyle='steps-mid',
)
ax.axvspan(
    xmin=unbiased.lights.inputs.time_start.ndarray.min().datetime,
    xmax=unbiased.lights.inputs.time_end.ndarray.max().datetime,
    alpha=0.3,
    label="lights",
)
ax.axvspan(
    xmin=unbiased.darks_up.inputs.time_start.ndarray.min().datetime,
    xmax=unbiased.darks_up.inputs.time_end.ndarray.max().datetime,
    alpha=0.3,
    label="darks",
    color="gray",
)
ax.axvspan(
    xmin=unbiased.darks_down.inputs.time_start.ndarray.min().datetime,
    xmax=unbiased.darks_down.inputs.time_end.ndarray.max().datetime,
    alpha=0.3,
    color="gray",
)
ax.set_xlabel("time (UTC)")
ax.set_ylabel("mean intensity (DN)")
ax.legend();

In [ ]:
unbiased.darks.to_jshtml()

In [ ]:
taps = unbiased.darks.taps

In [ ]:
axes_tap_xy = (taps.axis_tap_x, taps.axis_tap_y)
axis_tap_xy = "tap_xy"
tap_labels = taps.label.combine_axes(axes_tap_xy, axis_tap_xy)
taps = taps.combine_axes(axes_tap_xy, axis_tap_xy)

fig, ax = na.plt.subplots(
    axis_rows=axis_channel,
    axis_cols=axis_tap_xy,
    nrows=taps.shape[axis_channel],
    ncols=taps.shape[axis_tap_xy],
    sharex=True,
    sharey=True,
    constrained_layout=True,
    origin="upper",
)
na.plt.plot(
    taps.outputs.mean_trimmed(.01, (axis_time, axis_y)),
    axis=axis_x,
    ax=ax,
)
na.plt.set_ylim(
    bottom=-1,
    top=1,
    ax=ax,
)
na.plt.axvspan(
    xmin=0,
    xmax=taps.camera.sensor.num_blank,
    color="green",
    alpha=0.2,
    ax=ax,
    label="blank",
)
na.plt.axvspan(
    xmin=taps.num_x - taps.camera.sensor.num_overscan,
    xmax=taps.num_x,
    color="red",
    alpha=0.2,
    ax=ax,
    label="overscan",
)
na.plt.set_xlabel("columns", ax=ax[{level_0.axis_channel: ~0}])
na.plt.text(
    x=0.5,
    y=1.02,
    s=tap_labels[{axis_channel: 0}],
    ax=ax[{axis_channel: 0}],
    transform=na.plt.transAxes(ax[{level_0.axis_channel: 0}]),
    ha="center",
    va="bottom",
)
na.plt.text(
    x=1.02,
    y=0.5,
    s=level_0.channel,
    ax=ax[{axis_tap_xy: ~0}],
    transform=na.plt.transAxes(ax[{axis_tap_xy: ~0}]),
    ha="left",
    va="center",
);

In [ ]:
active = unbiased.active

In [ ]:
unbiased.shape

In [ ]:
active.shape

In [ ]:
electrons = active.electrons

In [ ]:
active.outputs.mean(axis_txy).ndarray

In [ ]:
electrons.outputs.mean(axis_txy).ndarray

In [ ]:
darks = electrons.darks.despiked

In [ ]:
dark = darks.mean(darks.axis_time)

In [ ]:
fig, ax = na.plt.subplots(
    axis_rows=axis_channel,
    nrows=dark.shape[axis_channel],
    sharex=True,
    sharey=True,
    constrained_layout=True,
    figsize=(5, 8),
    origin="upper",
)
na.plt.set_xlabel("detector $x$ (pix)", ax=ax[{axis_channel: ~0}])
na.plt.set_ylabel("detector $y$ (pix)", ax=ax)

norm = plt.Normalize(-5, 5)

colorizer = plt.Colorizer(norm=norm)

i = {darks.axis_time: 0}

na.plt.pcolormesh(
    dark[i].inputs.pixel.x,
    dark[i].inputs.pixel.y,
    C=dark[i].outputs,
    ax=ax,
    colorizer=colorizer,
)
na.plt.text(
    x=0.5,
    y=1.01,
    s=dark[i].channel,
    transform=na.plt.transAxes(ax),
    ax=ax,
    ha="center",
    va="bottom",
)
na.plt.set_aspect("equal", ax=ax)

plt.colorbar(
    mappable=plt.cm.ScalarMappable(colorizer=colorizer),
    ax=ax.ndarray,
    label=f"signal ({darks.outputs.unit:latex_inline})",
);

In [ ]:
dark_subtracted = electrons - dark.outputs

In [ ]:
residual = darks - dark.outputs

In [ ]:
hist = na.histogram(
    residual.taps.outputs, 
    axis=axis_txy,
    bins=dict(values=51),
    min=-20 * u.electron,
    max=+20 * u.electron,
    density=True,
)

In [ ]:
fig, ax = na.plt.subplots(
    axis_rows=axis_channel,
    nrows=4,
    sharex=True,
    figsize=(7, 6),
    constrained_layout=True,
    origin="upper"
)
na.plt.stairs(
    hist.inputs,
    hist.outputs,
    ax=ax,
    axis="values",
    label=residual.taps.label,
)
na.plt.text(
    x=0.01,
    y=.98,
    s=residual.channel,
    ax=ax,
    transform=na.plt.transAxes(ax),
    va="top",
)
ax_bottom = ax.ndarray[~0]
na.plt.axvline(0, ax=ax, color="black", linestyle="--")
na.plt.set_ylabel("probability density", ax=ax)
na.plt.set_xlabel(f"residual ({hist.inputs.unit:latex_inline})", ax=ax_bottom)
ax.ndarray[0].legend();

In [ ]:
residual.outputs.std(axis_txy).ndarray

In [ ]:
residual[{axis_channel: 2}].taps.outputs.std(axis_txy).ndarray

In [ ]:
lights = dark_subtracted.lights

In [ ]:
despiked = lights.despiked

In [ ]:
spikes = lights - despiked

In [ ]:
blink = na.stack(
    arrays=[
        lights.max(axis_time),
        despiked.max(axis_time),
        spikes.max(axis_time),
    ],
    axis="blink",
)[{axis_time: 0}]
blink.axis_time = "blink"
blink.to_jshtml(fps=1)

In [ ]:
where_spike = spikes.outputs >= .5 * u.electron

In [ ]:
scipy.stats.pearsonr(
    x=despiked.outputs[where_spike].ndarray,
    y=spikes.outputs[where_spike].ndarray, 
    axis=None,
).statistic

In [ ]:
level_1 = esis.data.Level_1(
    inputs=despiked.inputs,
    outputs=despiked.outputs,
    axis_time=axis_time,
    axis_channel=axis_channel,
    axis_x=axis_x,
    axis_y=axis_y,
)

In [ ]:
level_1.to_jshtml()